In [4]:
import numpy as np

In [80]:
class GeneNetwork:
    
    def __init__(self,name):
        self.name = name
        self.network = {}
        self.genes = None
        self.size = 0
        self.interactions = {}
        
    def add_genes(self, gene_dic):
        for gene in gene_dic.keys():
            kon, koff, srna, drna, sprot, dprot = gene_dic[gene]
            self.network[gene] = {'kon':kon,'koff':koff,'srna':srna, 'drna':drna, 'sprot':sprot, 'dprot':dprot}
            
            if not self.genes: #if there's no gene in the network yet
                self.genes = np.array(list(self.network.keys()))
                for g in self.genes:
                    if((gene,g) not in self.interactions.keys()):
                        self.interactions[(gene,g)] = 0

                    if((gene,g) not in self.interactions.keys()):
                        self.interactions[(g,gene)] = 0
            else:
                self.genes = np.append(self.genes, [gene])
                for g in self.genes:
                    if((gene,g) not in self.interactions.keys()):
                        self.interactions[(gene,g)] = 0

                    if((gene,name) not in self.interactions.keys()):
                        self.interactions[(g,gene)] = 0
            
            return self.genes
                        
                       
    def set_interactions(self,interaction_list): #interaction_list = [(source,target,coeff),(),()...etc]
        for source,target,value in interaction_list:
            self.interactions[(source,target)] = value
        return self.interactions
        
             
    def system_builder(self,init_cond,tf): #init_cond = {gene:(promoter state, rna, prot)}
        T = np.arange(0,tf,1/1000)
        prom = {}
        rna = {}
        prot = {}
        
        #Initialization of arrays and initial conditions.
        for gene in self.genes: 
            prom[gene] = np.array([0]*len(T)) #promoter states
            rna[gene] = np.array([0]*len(T)) #rna product
            prot[gene] = np.array([0]*len(T)) #protein
            
        #If no initial conditions specified: default (0,0,0)
            if gene in init_cond.keys():
                prom[gene][0] = init_cond[gene][0]
                rna[gene][0] = init_cond[gene][1]
                prot[gene][0] = init_çond[gene][2] #Peut-être utiliser des dictionnaires afin de faire init_cond[gene]['prot']
                
        #System running
        print("System running...")
        t  = 0
        i = 0
        while t < max(T):
            
            #Calculating all Ks to then calculate next reaction time
            Ks = {} #Dictinnary to keep all the ks
            
            for gene in self.genes:
                if not prom[gene][i]: #si le promoteur est éteint
                    kon = self.network[gene]['kon']
                    kon_t = kon #valeur calculée au temps t
                    for source,target in self.interactions.keys():
                        if target == gene:
                            coeff = self.interactions[(source,target)]
                            kon_t = kon_t + prot[source][i]*coeff*kon #on change la valeur du kon/off_t par QtéProt*Coeff*kon_original
                
                    Ks[gene] = kon_t
                    
                else: #si le promoteur est allumé
                    koff = self.network[gene]['koff']
                    koff_t = koff #valeur calculée au temps t
                    for source,target in self.interactions.keys():
                        if target == gene and self.interactions[(source,target)] != 0:
                            coeff = self.interactions[(source,target)]
                            koff_t = koff_t + prot[source][i]*-coeff*koff #-coeff car si inhibition du gène alors on augmente son koff
                    Ks[gene] = koff_t
            
            #Calculating next reaction time
            Ktot = sum(list(Ks.values()))
            tau = t + numpy.random.exponential(Ktot)%(1/1000)
            #Next reaction time is t+tau
            
            #Calculating system evolution until next reaction
            while t < tau:
                t += 1/1000
                i += 1
                
                for gene in self.genes:
                    G = self.network[gene] #we withdraw the gene informations
                    prom[gene][i] = prom[gene][i-1]
                    rna[gene][i] = rna[gene][i-1] + 1/1000*(prom[gene][i-1]*G['srna'] + rna[gene][i-1]*(G['drna']-G['sprot']))
                    if rna[gene][i] < 0: rna[gene][i]=0 
                    prot[gene][i] = prot[gene][i-1] + 1/1000*(rna[gene][i-1]*G['sprot'] + prot[gene][i-1]*(G['sprot']+G['dprot']))
                    if prot[gene][i] < 0: prot[gene][i] = 0 
            
            
            #Selecting promoter to switch.
            gene_list = self.genes
            p_genes = np.array([Ks[gene] for gene in gene_list])
            selected = np.random.choice(gene_list, p = p_genes)
            if prom[selected][i-1]:
                prom[selected][i] = 0
            else:
                prom[selected][i] = 1
            
        
        return T,prom,rna,prot
            

            

In [85]:
test_network = GeneNetwork('test')
test_network.add_genes({'gene1': (1,100,10,1,5,1),'gene2': (1,100,10,1,5,1)})
test_network.set_interactions([('gene1','gene1',0.2),('gene1','gene2',-0.4),('gene2','gene1',-0.4), ('gene2','gene2',0.2)])
init_cond = {}
test_network.system_builder(init_cond, 1000)



System running...


KeyError: 'gene2'

In [63]:
a = {'a':1}
print(np.array([a.keys()]))

[dict_keys(['a'])]
